In [1]:
# example_meta_data_reader.ipynb
# Authors: Stephan Meighen-Berger
# Shows how to deal with the output files

In [2]:
import awkward as ak
import pyarrow.parquet as pq

In [3]:
example_set = ak.from_parquet('./output/rset_1_meta_data.parquet')

In [4]:
example_set.fields

['event_id', 'mc_truth', 'photons_1', 'photons_2']

In [5]:
example_set.event_id

<Array [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] type='10 * int64'>

In [6]:
example_set.photons_1.fields

['sensor_id', 't', 'wave', 'sensor_hit_point', 'photon_dir']

In [7]:
len(example_set.photons_1.sensor_id)

10

In [8]:
# The hit doms for the second event
example_set.photons_1.sensor_id[1]

<Array [44, 54, 52, 49, 48, ... 47, 51, 54, 44] type='10 * int64'>

In [9]:
# The hit times for the second event
example_set.photons_1.t[1]

<Array [939, 2.96e+03, ... 1.01e+03, 868] type='10 * float64'>